<font color="orange" size=25> <br>Lesson 7 <br><br></font>
<font color="black" size=6>1. Introduction to GDAL<br><br></font>
<font color="black" size=6>2. Open a raster file with GDAL<br><br>3. Raster image visualization<br><br>4. Working with raster data<br><br>5. Map classification
 <br><br>
</font>

<font color="orange" size=6>2. Open a raster file with GDAL<br><br></font>
<font color="black" size=4>**Reading one Band**</font>

In [ ]:
#Import GDAL 
import os 
from osgeo import gdal 
import numpy as np
os.chdir(r'D:\M\地理大数据与云计算\实验数据\lesson7\lesson7_data\LC08_L1TP_120038_20190913_20190917_01_T1') 
band1_fn = 'LC08_L1TP_120038_20190913_20190917_01_T1_B2.TIF' 

#Open the band 1 GeoTIFF 
in_ds = gdal.Open(band1_fn) 
#请在此处输入代码

<font color="black" size=4>**Composite 3 bands**</font>

In [ ]:
band2_fn = 'LC08_L1TP_120038_20190913_20190917_01_T1_B3.TIF' 
band3_fn = 'LC08_L1TP_120038_20190913_20190917_01_T1_B4.TIF' 

#Create a three-band GeoTIFF with same properties as band 1 
gtiff_driver = gdal.GetDriverByName('GTiff') 
out_ds = gtiff_driver.Create('nat_color.tif', 
    in_band.XSize, in_band.YSize, 3, in_band.DataType) 
out_ds.SetProjection(in_ds.GetProjection()) 
out_ds.SetGeoTransform(in_ds.GetGeoTransform()) 

#Copy pixel data from input band to band 3 of output 
#请在此处输入代码
out_band = out_ds.GetRasterBand(3) 
out_band.WriteArray(in_data) 

#Copy pixel data from a dataset instead of a band 
in_ds = gdal.Open(band2_fn) 
out_band = out_ds.GetRasterBand(2) 
out_band.WriteArray(in_ds.ReadAsArray()) 
out_ds.GetRasterBand(1).WriteArray(gdal.Open(band3_fn).ReadAsArray()) 

<font color="black" size=4>**Compute statistics and build pyramids**</font>

In [ ]:
#Compute statistics on each output band 
out_ds.FlushCache() 
for i in range(1, 4): 
    out_ds.GetRasterBand(i).ComputeStatistics(False)
    
#Build over views/pyramid layers 
out_ds.BuildOverviews('average', [2, 4, 8, 16, 32]) 

<font color="black" size=4>**Reading partial datasets**</font>

In [ ]:
data = np.array(in_band.ReadAsArray(1400, 6000, 6, 3))

<font color="black" size=4>**Convert data type**</font>

In [ ]:
data = np.array(in_band.ReadAsArray(1400, 6000, 6, 3)).astype(float) 

<font color="black" size=4>**Read raster file properties**</font>

In [ ]:
# Check type of the variable 'raster' 
print(type(out_ds))

# Projection 
print(out_ds.GetProjection()) 

# Dimensions 
print(out_ds.RasterXSize) 
print(out_ds.RasterYSize) 

# Number of bands 
print(out_ds.RasterCount) 

# Metadata for the raster dataset 
print(out_ds.GetMetadata()) 

<font color="black" size=4>**Get raster bands**</font>

In [ ]:
# Read the raster band as separate variable 
band = out_ds.GetRasterBand(1) 

# Check type of the variable 'band' 
type(band) 

# Data type of the values 
gdal.GetDataTypeName(band.DataType) 

<font color="black" size=4>**Band statistics**</font>

In [ ]:
# Compute statistics if needed 
if band.GetMinimum() is None or band.GetMaximum()is None: 
    band.ComputeStatistics(0) 
print("Statistics computed.") 

# Fetch metadata for the band 
band.GetMetadata() 

# Print only selected metadata: 
print ("[ NO DATA VALUE ] = ", band.GetNoDataValue()) # none 
print ("[ MIN ] = ", band.GetMinimum()) 
print ("[ MAX ] = ", band.GetMaximum()) 

<font color="orange" size=6>3. Raster image visualization<br><br></font>
<font color="black" size=4>**Plot a raster band**</font>

In [ ]:
import matplotlib.pyplot as plt 

fig, ((ax0, ax1,ax2)) = plt.subplots(1, 3, figsize=(30, 10)) 
red = out_ds.GetRasterBand(3).ReadAsArray() 
green = out_ds.GetRasterBand(2).ReadAsArray() 
blue = out_ds.GetRasterBand(1).ReadAsArray() 
#请在此处输入代码
ax0.set_title('Red',fontsize =25) 
ax1.imshow(green) 
ax1.set_title('Green',fontsize =25) 
ax2.imshow(blue,cmap='gray') 
ax2.set_title('Blue (cmap=gray)',fontsize =25) 
plt.show() 

<font color="black" size=4>**Plot 3 raster bands**</font>

In [ ]:
import numpy as np

data = np.dstack((red, green, blue)) 
plt.imshow(data) 

In [ ]:
def stretch_data(data, num_stddev): 
    """Returns the data with a standard deviation stretch applied. 
    data - array containing data to stretch 
    num_stddev - number of standard deviations to use """ 
    mean = np.mean(data) 
    std_range = np.std(data) * 2 
    new_min = max(mean - std_range, np.min(data)) 
    new_max = min(mean + std_range, np.max(data)) 
    clipped_data = np.clip(data, new_min, new_max) 
    return clipped_data / (new_max - new_min) 

red_data = stretch_data(red,2) 
green_data = stretch_data(green,2) 
blue_data = stretch_data(blue,2) 
alpha = np.where(red_data + green_data + blue_data > 0, 1, 0) 
data = np.dstack((red_data, green_data, blue_data,alpha)) 
plt.imshow(data) 

<font color="black" size=4>**Plotting 3D data**</font>

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
from mpl_toolkits.mplot3d import Axes3D 
from osgeo import gdal 

ds = gdal.Open(r'D:\M\地理大数据与云计算\实验数据\lesson7\lesson7_data\DEM.tif') 
ov_band = ds.GetRasterBand(1) 
data = ov_band.ReadAsArray() 

# Calculate bounding coordinates. 
#请在此处输入代码 
minx = geotransform[0] 
maxy = geotransform[3] 
maxx = minx + ov_band.XSize * geotransform[1] 
miny = maxy + ov_band.YSize * geotransform[5] 

# Get the x and y arrays. 
x = np.arange(minx, maxx, geotransform[1]) 
y = np.arange(maxy, miny, geotransform[5]) 
x, y = np.meshgrid(x[:ov_band.XSize], y[:ov_band.YSize]) 

# Make the 3D plot. 
fig = plt.figure(figsize=(20, 10)) 
ax = fig.gca(projection='3d') 
ax.plot_surface(x, y, data, cmap='gist_earth', lw=0) 
plt.show() 

<font color="orange" size=6> 4. Working with raster data<br><br></font>
<font color="orange" size=5>**Histograms**</font>

<font color="black" size=4>**This code snippet shows the difference between approximate and exact values, using the classified elevation raster that we looked at earlier:**</font>

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
green = out_ds.GetRasterBand(2) 
hist = green.GetHistogram(0, 49868, 256, approx_ok=False) 
n_bins = np.linspace(256, 49868, 100) 
plt.hist(hist,n_bins) 

<font color="black" size=4>**Reprojecting images**</font>

In [ ]:
from osgeo import gdal, osr 
fig, ((ax0, ax1)) = plt.subplots(1, 2, figsize=(20, 10)) 

#Reprojecting images
input_path = r'D:\M\地理大数据与云计算\实验数据\lesson7\lesson7_data\LC08_L1TP_120038_20190913_20190917_01_T1\LC08_L1TP_120038_20190913_20190917_01_T1_B2.TIF' 
old_ds = gdal.Open(input_path, gdal.GA_ReadOnly)
srs = osr.SpatialReference() 
srs.SetWellKnownGeogCS('WGS84')
vrt_ds = gdal.AutoCreateWarpedVRT(old_ds, None, srs.ExportToWkt(),gdal.GRA_Bilinear) 
gdal.GetDriverByName('gtiff').CreateCopy('nat_color_wgs84.tif', vrt_ds) 

#plot 
old_ds_array = old_ds.ReadAsArray() 
vrt_ds_array = vrt_ds.ReadAsArray() 
ax0.imshow(old_ds_array) 
ax0.set_title('WGS_1984_UTM_Zone_50N',fontsize =25) 
ax1.imshow(vrt_ds_array) 
ax1.set_title('EPSG:4326 (WGS84)',fontsize =25) 
plt.show() 

<font color="black" size=4>**Clip raster by bounding box**</font>

In [ ]:
from osgeo import ogr, gdal 
InputImage = r'D:\M\地理大数据与云计算\实验数据\lesson7\lesson7_data\LC08_L1TP_120038_20190913_20190917_01_T1\LC08_L1TP_120038_20190913_20190917_01_T1_B3.TIF' 
Shapefile = r'D:\M\地理大数据与云计算\实验数据\lesson7\lesson7_data\nanjing.shp' 
RasterFormat = 'GTiff'
PixelRes = 30 
VectorFormat = 'ESRI Shapefile'

Raster = gdal.Open(InputImage, gdal.GA_ReadOnly) 
Projection = Raster.GetProjectionRef() 

VectorDriver = ogr.GetDriverByName(VectorFormat) 
VectorDataset = VectorDriver.Open(Shapefile, 0) # 0=Read-only, 1=Read-Write 

layer = VectorDataset.GetLayer() 
FeatureCount = layer.GetFeatureCount() 
print("Feature Count:",FeatureCount) 

In [ ]:
# Iterate through the shapefile features 
Count = 0
for feature in layer: 
    Count += 1 
    print("Processing feature "+str(Count)+" of "+str(FeatureCount)+"...") 
    
    # Get bounding box of the shapefile feature 
    geom = feature.GetGeometryRef() 
    minX, maxX, minY, maxY = geom.GetEnvelope() 
    
    # Create raster 
    OutTileName = str(Count)+'.SomeTileName.tif'
    OutTile = gdal.Warp(OutTileName, Raster, format=RasterFormat, outputBounds=[minX,    
                                      minY, maxX, maxY], xRes=PixelRes, yRes=PixelRes,  
                                      dstSRS=Projection, resampleAlg=gdal.GRA_NearestNeighbour, 
                                      options=['COMPRESS=DEFLATE']) 
print("Done.") 
plt.figure(figsize=(20, 10)) 
OutTile_array = OutTile.ReadAsArray() 
plt.imshow(OutTile_array) 
plt.show() 

<font color="black" size=4>**Mosaic multiple images together**</font>

In [ ]:
import glob 
import math 
import os 
from osgeo import gdal, osr 

# The get_extent function from the text is in ch10funcs.py. 
def get_extent(fn): 
   #Returns min_x, max_y, max_x, min_y
   ds = gdal.Open(fn) 
   gt = ds.GetGeoTransform() 
   return (gt[0], gt[3], gt[0] + gt[1] * ds.RasterXSize, 
	  gt[3] + gt[5] * ds.RasterYSize) 

# Don't forget to change the directory. 
os.chdir(r'D:\M\地理大数据与云计算\实验数据\lesson7\lesson7_data\Massachusetts') 

#Get the list of tiffs that start with O. 
in_files = glob.glob('O*.tif') 

In [ ]:
# Loop through all of the files and get the bounding coordinates for the
# whole batch. This will be the output extent.
min_x, max_y, max_x, min_y = get_extent(in_files[0])
for fn in in_files[1:]:
    minx, maxy, maxx, miny = get_extent(fn)
    min_x = min(min_x, minx)
    max_y = max(max_y, maxy)
    max_x = max(max_x, maxx)
    min_y = min(min_y, miny)

# Calculate the dimensions for the output based on the output extent.
in_ds = gdal.Open(in_files[0])
gt = in_ds.GetGeoTransform()
rows = math.ceil((max_y - min_y) / -gt[5])
columns = math.ceil((max_x - min_x) / gt[1])

# Create the output dataset.
driver = gdal.GetDriverByName('gtiff')
out_ds = driver.Create('mosaic2.tif', columns, rows)
out_ds.SetProjection(in_ds.GetProjection())
out_band = out_ds.GetRasterBand(1)

In [ ]:
# Change the upper left coordinates in the geotransform and add it to the
# output image.
gt = list(in_ds.GetGeoTransform())
gt[0], gt[3] = min_x, max_y
out_ds.SetGeoTransform(gt)

# Loop through the input files.
for fn in in_files:
    in_ds = gdal.Open(fn)

    # Create a transformer between this input image and the output mosaic
    # and then use it to calculate the offsets for this raster in the
    # mosaic.
    
    #请在此处输入代码
    success, xyz = trans.TransformPoint(False, 0, 0)
    x, y, z = map(int, xyz)

    # Copy the data.
    data = in_ds.GetRasterBand(1).ReadAsArray()
    out_band.WriteArray(data, x, y)

plt.figure(figsize=(10, 10))
OutTile_array = out_band.ReadAsArray()
plt.imshow(OutTile_array)
plt.show()

<font color="orange" size=6><br>4. Map algebra<br><br></font>
<font color="black" size=4>**Local analysis**</font>

In [ ]:
def make_raster(in_ds, fn, data, data_type, nodata=None): 
    """Create a one-band GeoTIFF. in_ds - datasource to copy projection and  
    geotransform from fn - path to the file to create data - NumPy array containing data   
    to write data_type - output data type nodata - optional NoData value """ 
    #请在此处输入代码
    out_ds = driver.Create( fn, in_ds.RasterXSize, in_ds.RasterYSize, 1, data_type) 
    out_ds.SetProjection(in_ds.GetProjection()) 
    out_ds.SetGeoTransform(in_ds.GetGeoTransform()) 
    out_band = out_ds.GetRasterBand(1) 
    if nodata is not None: 
        out_band.SetNoDataValue(nodata) 
        out_band.WriteArray(data) 
        out_band.FlushCache() 
        out_band.ComputeStatistics(False) 
    return out_ds 

In [ ]:
def compute_overview_levels(band): 
    """Return an appropriate list of overview levels.""" 
    max_dim = max(band.XSize, band.YSize) 
    overviews = [] 
    level = 1 
    while max_dim > 256: 
        level *= 2 
        overviews.append(level) 
        max_dim /= 2 
    return overviews 

In [ ]:
import os 
import numpy as np 
from osgeo import gdal 
import matplotlib.pyplot as plt

os.chdir(r'D:\M\地理大数据与云计算\实验数据\lesson7\lesson7_data\LC08_L1TP_120038_20190913_20190917_01_T1') 
band_green_fn = 'LC08_L1TP_120038_20190913_20190917_01_T1_B2.TIF' 
in_ds_green = gdal.Open(band_green_fn) 
band_nir_fn = 'LC08_L1TP_120038_20190913_20190917_01_T1_B5.TIF'
in_ds_nir = gdal.Open(band_nir_fn) 
out_fn = r'D:\M\地理大数据与云计算\实验数据\lesson7\lesson7_data\ndwi.tif' 
green = in_ds_green.GetRasterBand(1).ReadAsArray().astype(np.float) 
nir = in_ds_nir.GetRasterBand(1).ReadAsArray() 
#Mask the green band 
green = np.ma.masked_where(nir + green == 0, green) 
#Calculate NDWI 
#请在此处输入代码
#Fill the empty cells 
ndwi = ndwi.filled(-99) 
out_ds = make_raster(in_ds_green, out_fn, ndwi, gdal.GDT_Float32, -99) 
overviews = compute_overview_levels(out_ds.GetRasterBand(1)) 
out_ds.BuildOverviews('average', overviews) 


In [ ]:
plt.imshow(ndwi,cmap='Greys',vmin=-1, vmax=1) 
plt.colorbar() 
plt.show() 

<font color="black" size=4>**Focal analyses – make slices**</font>

In [ ]:
def make_slices(data, win_size): 
    """Return a list of slices given a window size. 
    data - two-dimensional array to get slices from win_size 
    - tuple of (rows, columns) for the moving window """ 
    rows = data.shape[0] - win_size[0] + 1 
    cols = data.shape[1] - win_size[1] + 1 
    slices = [] 
    for i in range(win_size[0]): 
        for j in range(win_size[1]): 
            slices.append(data[i:rows+i, j:cols+j]) 
    return slices 

<font color="black" size=4>**Focal analyses – smooth an elevation dataset**</font>

In [ ]:
import os 
import numpy as np 
from osgeo import gdal 
import matplotlib.pyplot as plt 
in_fn = r'D:\M\地理大数据与云计算\实验数据\lesson7\lesson7_data\DEM.tif' 
out_fn = r'D:\M\地理大数据与云计算\实验数据\lesson7\lesson7_data\smooth_DEM.tif'
in_ds = gdal.Open(in_fn) 
in_band = in_ds.GetRasterBand(1) 
in_data = in_band.ReadAsArray() 
slices = make_slices(in_data, (3, 3)) 
stacked_data = np.ma.dstack(slices) 
rows, cols = in_band.YSize, in_band.XSize 
out_data = np.ones((rows, cols), np.float32) * -99 
out_data[1:-1, 1:-1] = np.mean(stacked_data, 2)
OutTile = make_raster(in_ds, out_fn, out_data, gdal.GDT_Float32, -99) 
fig, ((ax0, ax1)) = plt.subplots(1, 2, figsize=(20, 10)) 
ax0.imshow(in_data) 
ax0.set_title('Raw DEM',fontsize =25) 

OutTile_array = OutTile.ReadAsArray() 

ax1.imshow(OutTile_array) 
ax1.set_title('Smoothing (mean)',fontsize =25) 
plt.show() 

<font color="black" size=4>**Focal analyses – calculate slope**</font>

In [ ]:
import os 
import numpy as np 
from osgeo import gdal 
in_fn = r"D:\M\地理大数据与云计算\实验数据\lesson7\lesson7_data\DEM.tif" 
out_fn = r"D:\M\地理大数据与云计算\实验数据\lesson7\lesson7_data\slope.tif" 

in_ds = gdal.Open(in_fn) 
cell_width = in_ds.GetGeoTransform()[1] 
cell_height = in_ds.GetGeoTransform()[5] 
band = in_ds.GetRasterBand(1) 
in_data = band.ReadAsArray().astype(np.float) 

#Initialize output array with -99 
out_data = np.ones((band.YSize, band.XSize)) * -99 

slices = make_slices(in_data, (3, 3)) 
rise = ((slices[6] + (2 * slices[7]) + slices[8]) - (slices[0] + (2 * slices[1]) + slices[2])) /(8 * cell_height) 
run = ((slices[2] + (2 * slices[5]) + slices[8]) - (slices[0] + (2 * slices[3]) + slices[6])) /(8 * cell_width) 
dist = np.sqrt(np.square(rise) + np.square(run)) 

#Output edges don’t get slope data 
out_data[1:-1, 1:-1] = np.arctan(dist) * 180 / np.pi 
OutTile = make_raster(in_ds, out_fn, out_data, gdal.GDT_Float32, -99) 
plt.figure(figsize=(10, 10)) 
OutTile_array = OutTile.ReadAsArray() 
im=plt.imshow(OutTile_array,vmin=0, vmax=50) 
plt.colorbar(im) 
plt.show() 


<font color="black" size=4>**Zonal analyses**</font>

In [ ]:
import numpy as np 
import scipy.stats 
from osgeo import gdal 

def get_bins(data): 
    """Return bin edges for all unique values in data.""" 
    bins = np.unique(data) 
    return np.append(bins, max(bins) + 1) 

landcover_fn = r'D:\M\地理大数据与云计算\实验数据\lesson7\lesson7_data\Utah\landcover60.tif' 
ecoregion_fn = r'D:\M\地理大数据与云计算\实验数据\lesson7\lesson7_data\Utah\utah_ecoIII60.tif' 
out_fn = r'D:\M\地理大数据与云计算\实验数据\lesson7\lesson7_data\Utah\histogram.csv'
eco_ds = gdal.Open(ecoregion_fn) 
eco_band = eco_ds.GetRasterBand(1) 
eco_data = eco_band.ReadAsArray().flatten() 
eco_bins = get_bins(eco_data) 

In [ ]:
lc_ds = gdal.Open(landcover_fn) 
lc_band = lc_ds.GetRasterBand(1) 
lc_data = lc_band.ReadAsArray().flatten() 
lc_bins = get_bins(lc_data) 

#Compute histogram 
hist, eco_bins2, lc_bins2, bn = scipy.stats.binned_statistic_2d( eco_data, lc_data, 
    lc_data, 'count', [eco_bins, lc_bins]) 
#Add bin info to histogram 
hist = np.insert(hist, 0, lc_bins[:-1], 0) 
row_labels = np.insert(eco_bins[:-1], 0, 0) 
hist = np.insert(hist, 0, row_labels, 1) 

#Save output 
np.savetxt(out_fn, hist, fmt='%1.0f', delimiter=',') 

In [ ]:
import pandas as pd 
dataFrame = pd.read_csv(r'D:\M\地理大数据与云计算\实验数据\lesson7\lesson7_data\Utah\histogram.csv') 
dataFrame 


<font color="black" size=4>**Global analyses**<br><br></font>
<font color="black" size=4>**Let’s calculate average distance to roads within a wilderness area**</font>

In [ ]:
import os 
import sys 
from osgeo import gdal, ogr 
folder = r'D:\M\地理大数据与云计算\实验数据\lesson7\lesson7_data\Idaho' 
roads_ln = 'allroads'
wilderness_ln = 'wilderness'
road_raster_fn = 'church_roads.tif'
proximity_fn = 'proximity.tif'
#Set the cell size for the analysis 
cellsize = 10 
#Get extent of wilderness area 
shp_ds = ogr.Open(folder) 
wild_lyr = shp_ds.GetLayerByName(wilderness_ln) 
wild_lyr.SetAttributeFilter( "WILD_NM = 'Frank Church - RONR' ") 
envelopes = [row.geometry().GetEnvelope() for row in wild_lyr] 
coords = list(zip(*envelopes)) 
minx, maxx = min(coords[0]), max(coords[1]) 
miny, maxy = min(coords[2]), max(coords[3]) 
#Select roads in wilderness extent 
road_lyr = shp_ds.GetLayerByName(roads_ln) 
road_lyr.SetSpatialFilterRect(minx, miny, maxx, maxy) 

In [ ]:
os.chdir(folder) 
tif_driver = gdal.GetDriverByName('GTiff') 
cols = int((maxx - minx) / cellsize) 
rows = int((maxy - miny) / cellsize) 
#Create empty raster for roads 
road_ds = tif_driver.Create(road_raster_fn, cols, rows) 
road_ds.SetProjection( road_lyr.GetSpatialRef().ExportToWkt()) 
road_ds.SetGeoTransform( (minx, cellsize, 0, maxy, 0, -cellsize)) 

def progress_callback(complete, message, unknown): 
    return 1 

#Burn roads into raster 
gdal.RasterizeLayer( road_ds, [1], road_lyr, burn_values=[1], 
                                  callback=progress_callback ) 

#Burn proximity data into new raster 
prox_ds = tif_driver.Create( proximity_fn, cols, rows, 1, gdal.GDT_Int32) 
prox_ds.SetProjection(road_ds.GetProjection()) 
prox_ds.SetGeoTransform(road_ds.GetGeoTransform()) 
gdal.ComputeProximity( road_ds.GetRasterBand(1), prox_ds.GetRasterBand(1), 
                                       ['DISTUNITS=GEO'], progress_callback) 


In [ ]:
#Burn wilderness into temporary raster 
wild_ds = gdal.GetDriverByName('MEM').Create( 'tmp', cols, rows) 
wild_ds.SetProjection(prox_ds.GetProjection()) 
wild_ds.SetGeoTransform(prox_ds.GetGeoTransform()) 
gdal.RasterizeLayer(wild_ds, [1], wild_lyr, burn_values=[1],callback=progress_callback) 

#Set NoData outside wilderness 
wild_data = wild_ds.ReadAsArray() 
prox_data = prox_ds.ReadAsArray() 
prox_data[wild_data == 0] = -99 
prox_ds.GetRasterBand(1).WriteArray(prox_data) 
prox_ds.GetRasterBand(1).SetNoDataValue(-99) 
prox_ds.FlushCache() 

stats = prox_ds.GetRasterBand(1).ComputeStatistics( False, progress_callback)
print('Mean distance from roads is', stats[2]) 

<font color="orange" size=6><br> 5. Map classification<br><br></font>
<font color="black" size=4>**K-means clustering**</font>

In [ ]:
import os 
import numpy as np 
import spectral 
from osgeo import gdal 
import matplotlib.pyplot as plt 
def stack_bands(filenames): 
    """Returns a 3D array containing all band data from all files.""" 
    bands = [] 
    for fn in filenames: 
        ds = gdal.Open(fn) 
        for i in range(1, ds.RasterCount + 1): 
            bands.append(ds.GetRasterBand(i).ReadAsArray()) 
    return np.dstack(bands) 

folder = r'D:\M\地理大数据与云计算\实验数据\lesson7\lesson7_data\Utah'
raster_fns = ['LE70380322000181EDC02_60m.tif', 'LE70380322000181EDC02_TIR_60m.tif'] 
out_fn = r'D:\M\地理大数据与云计算\实验数据\lesson7\lesson7_data\Utah\kmeans_prediction_60m.tif' 
os.chdir(folder) 

In [ ]:
#Run the model 
data = stack_bands(raster_fns) 
#请在此处输入代码

ds = gdal.Open(raster_fns[0]) 
out_ds = make_raster(ds, out_fn, classes, gdal.GDT_Byte) 
levels = compute_overview_levels(out_ds.GetRasterBand(1)) 
out_ds.BuildOverviews('NEAREST', levels) 
out_ds.FlushCache()
out_ds.GetRasterBand(1).ComputeStatistics(False) 

fig, ((ax0, ax1)) = plt.subplots(1, 2, figsize=(20, 10)) 
ax0.imshow(data) 
ax0.set_title('Raw Image',fontsize =25) 
ax1.imshow(classes) 
ax1.set_title('K-means classification',fontsize =25) 
plt.show() 

<font color="black" size=4>**Supervised classification using CART**</font>

In [ ]:
import csv 
import os 
import numpy as np 
from sklearn import tree 
from osgeo import gdal 

folder = r'D:\M\地理大数据与云计算\实验数据\lesson7\lesson7_data\Utah' 
raster_fns = ['LE70380322000181EDC02_60m.tif', 'LE70380322000181EDC02_TIR_60m.tif'] 
out_fn = 'tree_prediction60.tif' 
train_fn = r'D:\M\地理大数据与云计算\实验数据\lesson7\lesson7_data\Utah\training_data.csv' 
gap_fn = r'D:\M\地理大数据与云计算\实验数据\lesson7\lesson7_data\Utah\landcover60.tif' 

os.chdir(folder) 

In [ ]:
# Read the coordinates and actual classification from the csv.
# This is the training data.
xys = []
classes = []
with open(train_fn) as fp:
    reader = csv.reader(fp)
    next(reader)
    for row in reader:
        xys.append([float(n) for n in row[:2]])
        classes.append(int(row[2]))

# Calculate the pixel offsets for the coordinates obtained from the csv.
ds = gdal.Open(raster_fns[0])
pixel_trans = gdal.Transformer(ds, None, [])
offset, ok = pixel_trans.TransformPoints(True, xys)
cols, rows, z = zip(*offset)

# Get the satellite data. 
data = stack_bands(raster_fns) 
rows=np.array(rows,dtype=np.int32) 
cols=np.array(cols,dtype=np.int32) 

In [ ]:
# Sample the satellite data at the coordinates from the csv. 
sample = data[rows, cols, :] 

# Fit the classification tree. 
clf = tree.DecisionTreeClassifier(max_depth=5) 
clf = clf.fit(sample, classes) 

# Apply the new classification tree model to the satellite data. 
rows, cols, bands = data.shape 
data2d = np.reshape(data, (rows * cols, bands)) 
#请在此处输入代码 
prediction = np.reshape(prediction, (rows, cols)) 

# Set the pixels with no valid satellite data to 0. 
prediction[np.sum(data, 2) == 0] = 0 

# Save the output. 
predict_ds = make_raster(ds, out_fn, prediction, gdal.GDT_Byte, 0) 
predict_ds.FlushCache() 
levels = compute_overview_levels(predict_ds.GetRasterBand(1)) 
predict_ds.BuildOverviews('NEAREST', levels) 

In [ ]:
# Apply the color table from the SWReGAP landcover raster. 
gap_ds = gdal.Open(gap_fn) 
colors = gap_ds.GetRasterBand(1).GetRasterColorTable() 
predict_ds.GetRasterBand(1).SetRasterColorTable(colors) 

plt.figure(figsize=(10, 10)) 
OutTile_array = predict_ds.ReadAsArray() 
plt.imshow(OutTile_array) 
plt.colorbar() 
plt.show() 

<font color="black" size=4>**Accuracy assessments**</font>

In [ ]:
import csv 
import os 
import numpy as np 
from sklearn import metrics 
import skll 
from osgeo import gdal 

folder = r'D:\M\地理大数据与云计算\实验数据\lesson7\lesson7_data\Utah' 
accuracy_fn = 'accuracy_data.csv' 
matrix_fn = 'confusion_matrix.csv' 
prediction_fn = r'D:\M\地理大数据与云计算\实验数据\lesson7\lesson7_data\Utah\tree_prediction60.tif' 

os.chdir(folder) 

# Collect the data needed for the accuracy assessment. 
xys = []
classes = []
with open(accuracy_fn) as fp:
    reader = csv.reader(fp)
    next(reader)
    for row in reader:
        xys.append([float(n) for n in row[:2]])
        classes.append(int(row[2]))

In [ ]:
ds = gdal.Open(prediction_fn) 
pixel_trans = gdal.Transformer(ds, None, []) 
offset, ok = pixel_trans.TransformPoints(True, xys) 
cols, rows, z = zip(*offset) 

data = ds.GetRasterBand(1).ReadAsArray() 
rows=np.array(rows,dtype=np.int32) 
cols=np.array(cols,dtype=np.int32) 
sample = data[rows, cols] 

# Compute kappa. 
#请在此处输入代码 

# Create the confusion matrix. 
labels = np.unique(np.concatenate((classes, sample))) 
matrix = metrics.confusion_matrix(classes, sample, labels) 

# Add labels to the matrix and save it. 
matrix = np.insert(matrix, 0, labels, 0) 
matrix = np.insert(matrix, 0, np.insert(labels, 0, 0), 1) 
np.savetxt(matrix_fn, matrix, fmt='%1.0f', delimiter=',') 

In [ ]:
import pandas as pd 
dataFrame = pd.read_csv(r'D:\M\地理大数据与云计算\实验数据\lesson7\lesson7_data\Utah\confusion_matrix.csv') 
dataFrame.head(5)